In [3]:
!pip install googlemaps

In [4]:
import googlemaps
import time

In [5]:
k='API key'   ### import google map API key

In [47]:

gmaps = googlemaps.Client(key=k)

params = {
    'query': ["carer support group"],     ### search content 
    'location': (-37.840935, 144.946457),  ### default location
    'radius': 200000                       ### search radius
}

In [48]:
x = gmaps.places(**params) 

In [49]:
len(x['results'])      ### display how many search record

17

In [50]:
x['results'][0]["name"]

'Carers Victoria'

In [51]:
for i in range(len(x['results'])):                 ### List the record name
    print(x['results'][i]["name"]) 

Carers Victoria
Sunspec Support Group for Parents & Carers of Children with Special Needs
Stroke Association Of Victoria
Fronditha Care Support Hub
Boroondara Stroke Support Group
Uniting lifeAssist
Omega Support Group Pty. Ltd.
Interchange Outer East
Life Without Limits Advanced Support Group (Life Without Limits)
Autism Plus
ONCALL Group
Parkinsons Vic
IERA Social Support Group
Action on Disability Within Ethnic Communities
Banksia Palliative Care Service Inc.
Amaze
CO.AS.IT.


In [16]:
x['results']

[{'formatted_address': '98 Riversdale Rd, Hawthorn VIC 3122, Australia',
  'geometry': {'location': {'lat': -37.8290399, 'lng': 145.0349228},
   'viewport': {'northeast': {'lat': -37.82758502010728,
     'lng': 145.0362920798927},
    'southwest': {'lat': -37.83028467989273, 'lng': 145.0335924201073}}},
  'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/generic_business-71.png',
  'id': 'd47a091f0b50fbaa0443632b10e766827c96f565',
  'name': 'Dementia Australia',
  'opening_hours': {'open_now': True},
  'photos': [{'height': 2160,
    'html_attributions': ['<a href="https://maps.google.com/maps/contrib/112401115369754735252/photos">A Google User</a>'],
    'photo_reference': 'CmRaAAAAOuVrAKW3Osg_phEcucYakPxVShGn72gMqz7xSxR8LwV0_idjJziW9j5rcIzV-qZpau1q_S43gYlDkNqImHBo5HBGdb8hTMX3VVRcFazv2USwgy4atwio27GbvFAUel0oEhBBaBsr8qSZ5jt-CCd7vV6PGhTEKIMWfuFFacmDajMvUsmSMlylTA',
    'width': 3840}],
  'place_id': 'ChIJzwG8XTpC1moRGN8E52cB4IY',
  'plus_code': {'compound_code': '52CM+9X Hawtho

In [53]:
params['page_token'] = x['next_page_token']

time.sleep(2)
x1 = gmaps.places(**params)

In [54]:
len(x1['results'])

20

In [55]:
x1['results']

[{'formatted_address': '135 David St, Dandenong VIC 3175, Australia',
  'geometry': {'location': {'lat': -37.9765911, 'lng': 145.2184308},
   'viewport': {'northeast': {'lat': -37.97548277010728,
     'lng': 145.2198290798927},
    'southwest': {'lat': -37.97818242989272, 'lng': 145.2171294201073}}},
  'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/doctor-71.png',
  'id': '0ccd0ee6e625f5ffe9abee3ad93bc6d6f5f23d8e',
  'name': 'Dandenong Hospital',
  'opening_hours': {'open_now': True},
  'photos': [{'height': 3120,
    'html_attributions': ['<a href="https://maps.google.com/maps/contrib/110313118864890612948/photos">Steve D</a>'],
    'photo_reference': 'CmRaAAAAo_miH4RexqdY9OMCxuvl6wlueo4g7QkpdE-YdR94qnkS1DoiNQTMqsEZZXWVJCTIjLF5XstgP5cs9HE41YqjeDve6btcTm45NEzwUBX1sqpSix48BW1gVCjqQcMYGVeqEhDAp9QOJBLBY4H4P7E8eoEBGhRhzpDpEAyMw9XT76dnxn7BFBOY9Q',
    'width': 4160}],
  'place_id': 'ChIJo7wBCRkU1moRldoRKSnG6r4',
  'plus_code': {'compound_code': '26F9+99 Dandenong, Victoria',
   

In [62]:
params['page_token'] = x1['next_page_token']

time.sleep(2)
x2 = gmaps.places(**params)

In [63]:
len(x2['results'])

20

In [64]:
x2['results']

[{'formatted_address': '462 Neerim Rd, Murrumbeena VIC 3163, Australia',
  'geometry': {'location': {'lat': -37.8897313, 'lng': 145.0676339},
   'viewport': {'northeast': {'lat': -37.88844927010727,
     'lng': 145.0689348798927},
    'southwest': {'lat': -37.89114892989272, 'lng': 145.0662352201073}}},
  'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/doctor-71.png',
  'id': '08d7ed11591c2a0abd7daa0d5e4c85b84ddad540',
  'name': 'DBT',
  'place_id': 'ChIJBwWxpu1r1moRQYcLCT6vYRs',
  'plus_code': {'compound_code': '4369+43 Murrumbeena, Victoria',
   'global_code': '4RJ74369+43'},
  'rating': 0,
  'reference': 'ChIJBwWxpu1r1moRQYcLCT6vYRs',
  'types': ['hospital', 'point_of_interest', 'establishment'],
  'user_ratings_total': 0},
 {'formatted_address': '5 Studley Ave, Kew VIC 3101, Australia',
  'geometry': {'location': {'lat': -37.8067154, 'lng': 145.0235856},
   'viewport': {'northeast': {'lat': -37.80527857010728,
     'lng': 145.0252062298927},
    'southwest': {'lat': -37.